# Data Collection

## Combining dataset
Do yêu cầu dataset phải có tối thiểu 1000 dòng $\rightarrow$ nhóm quyết định gộp 2 file `.csv` trong tập dataset **Student Alcohol Consumption** gồm `student-mat.csv` và `student-por.csv` (2 file đều có chung các thuộc tính) thành `student-combine.csv` (có thêm thuộc tính `subject` cho `mat` và `por`) để xử lý cho các bước kế tiếp

In [1]:
import pandas as pd

df_mat = pd.read_csv("../data/raw/student-mat.csv")
df_por = pd.read_csv("../data/raw/student-por.csv")

In [2]:
# Thêm giá trị cho thuộc tính subject
df_mat['subject'] = 'mat'
df_por['subject'] = 'por'

In [3]:
# Gộp 2 data frame
df_combine = pd.concat([df_mat, df_por], axis=0, ignore_index=True)

df_combine.to_csv("../data/raw/student-combine.csv", index=False)

## Why choose this dataset?
- Nhóm chọn dataset **Student Alcohol Consumption** vì tính thực tiễn và dữ liệu đa dạng. Dataset giúp người đọc có một cái nhìn khách quan về các yếu tố tác động, từ môi trường đến hoạt động hàng ngày, có ảnh hưởng như thế nào đến hành vi tiêu thụ cồn. Từ đó, nhóm có thể phân tích và đánh giá tác động thực tế của lối sống lên hiệu suất học tập (môn toán và môn tiếng Bồ Đào Nha) của học sinh trung học.

- Một số câu hỏi và insight tiềm năng có thể tìm thấy ở dataset này:
    1. Câu hỏi về hiệu suất học tập: mức độ tiêu thụ cồn (hàng ngày và cuối tuần) có mối liên hệ như thế nào đến kết quả học tập hay không?

    2. Câu hỏi về môi trường và xã hội: liệu học sinh dành nhiều thời gian đi chơi, hay ở trong một mối quan hệ tình cảm thường sẽ tiêu thụ cồn nhiều hơn hay không?

    3. Câu hỏi về gia đình: trình độ học vấn của ba, mẹ và sự ổn định trong gia đình có ảnh hưởng đến hành vi uống rượu của con cái hay không?

    4. Câu hỏi về sức khoẻ, đi học: tình trạng sức khoẻ hay số ngày nghỉ học có phản ánh sớm việc sa sút trong học tập?

# Data Exploration

## Categorical Columns Analysis

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import math
import pandas as pd

In [ ]:
df = pd.read_csv("../data/raw/student-combine.csv")

In [ ]:
cat_cols = df.select_dtypes(include=['object']).columns
print(f"Categorical colums: {cat_cols}")

### Value Distribution

In [ ]:
def analyze_cat_col(colname, df):
    print(f"\nCategorical column: {colname.upper()}")

    unique_count = df[colname].nunique()
    print(f"- Unique value: {unique_count}")
    
    val_counts = df[colname].value_counts()
    top_5 = val_counts.head(5)
    
    print(f"- Top 5 most frequent value:\n {top_5}")
    
    print(f"- Percentage distribution:")
    distributions = df[colname].value_counts(normalize= True) * 100
    for uni, val in distributions.items():
        print(f"{uni} : {val:.2f}%")

In [ ]:
for col in cat_cols:
    analyze_cat_col(col, df)

In [ ]:
# Visualization cho mỗi cột kiểu category
n_cols = 3
n_rows = math.ceil(len(cat_cols) / n_cols)
fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, 4 * n_rows))
axes = axes.flatten() # Làm phẳng mảng axes

for i, colname in enumerate(cat_cols):
    sns.countplot(data=df, x=colname, ax=axes[i], palette="viridis", hue=colname)
    
    axes[i].set_title(f'Distribution of \"{colname}\"')
    axes[i].set_xlabel('')

    for container in axes[i].containers:
        axes[i].bar_label(container)

# Tắt các ô trống nếu số biểu đồ không lấp đầy lưới
for i in range(len(cat_cols), len(axes)):
    fig.delaxes(axes[i])

plt.tight_layout()
plt.show()

**Nhận xét:**
- Một số cột bị mất cân bằng cao (`Pstatus`, `schoolsup`, `higher`): `Pstatus = T` (88%), `schoolsup = no` (88%), `higher = yes` (90%) $\rightarrow$ Những cột này sẽ ít có khả năng phân loại vì không có sự biến thiên mà chỉ thiên về 1 phía. Gây ra Model bias nếu sử dụng làm `Target`

### Data Quality

In [ ]:
missing_vals = df[cat_cols].isnull().sum()
print(f"Missing value in categorical columns:\n{missing_vals}")

**Nhận xét:** 
- Không có cột nào có giá trị bị thiếu, giá trị bất thường
- Tuy nhiên, đối với cột `Fjob` ta thấy `health` chỉ chiếm số lượng ít (3.93%) và kích thước mẫu của nhóm khá nhỏ (n = 41) nhưng vẫn đủ ngưỡng tối thiểu để thực hiện các so sánh thống kê

### **Nhận xét chung:** 
- Các cột dữ liệu category: `school`, `sex`, `address`, `famsize`, `Pstatus`, `Mjob`, `Fjob`, `reason`, `guardian`, `schoolsup`, `famsup`, `paid`, `activities`, `nursery`, `higher`, `internet`, `romantic`, `subject`
- Các cột đều bị mất cân bằng dữ liệu (trừ cột `sex`, `activities` khá cân bằng)
- Các cột không có các giá trị bất thường, lỗi hay các giá trị hiếm (trừ `Fjob` nhưng nhóm quyết định giữ giá trị hiếm lại mà không gộp chung)

## Initial Observation and Insights

### **Summary**
**1. Key observation:**
- Sự bất thường của Target (`G3`): điểm số có phân phối gần chuẩn nhưng có 53 trường hợp xuất hiện điểm bằng 0. Kèm theo đó là khi `G3` = 0 thì `absences` = 0, điều này là khá vô lý vì theo hành vi học tập, nếu không nghỉ buổi nào nhưng có `G3` = 0. Bên cạnh đó, ta thấy `G1`, `G2` đa số > 0 nếu `G3` = 0  
$\rightarrow$ Có thể là những học sinh này đã bỏ thi, học lực kém, hoặc do các yếu tố tác động (hành vi uống rượu (?)) khiến cho `G3` bằng 0

- Phân phối lệch của `absences`: bị lệch phải nhiều (Skewness = 3.74). Trung vị là 2 nhưng xuất hiện các outlier rất lớn $\rightarrow$ Điều này cho thấy việc các học sinh có xu hướng ít nghỉ học, có 1 nhóm nhỏ nghỉ học nhiều.

- Sự đồng nhất về nhân khẩu học: có sự mất cân bằng lớn trong các biến: 90% có cha mẹ sống chung (`PStatus` = T), đa số có nguyện vọng học cao hơn (`higher` = yes) và có tham gia các lớp học thêm (`schoolsup` = yes)

**2. Data quality issues:**
- Imbalanced class: các cột category nhưng đã phân tích ở trên `Pstatus`, `higher`, `Fjob` bị mất cân bằng nghiêm trọng $\rightarrow$ giảm độ tin cậy khi phân tích hành vi của các nhóm này

**3. Necessary preprocessing steps:**
- Data Integration: Gộp hai bộ dữ liệu student-mat và student-por. Thêm cột định danh Subject ('Math'/'Por') để phân biệt ngữ cảnh môn học.

- Data cleaning:

- Feature Engineering:
    - Tạo `Total_alc` dựa vào `Dalc` và `Walc`

- Encoding: 
    - Sử dụng **Binary Encoding** cho các biến nhị phân (`school`, `sex`, `address`, `famsize`, ... )
    - Sử dụng **One-Hot Encoding** cho các biến có nhiều nhóm (`Mjob`, `Fjob`, `reason`, `guardian`, ... )

**4. Interesting patterns could lead to research questions:**
- Những học sinh có số `absences` cao thì sẽ tiêu thụ cồn (`Dalc`/`Walc`) nhiều nhất?

- Sự tương quan về quá trình học tập `G1`, `G2`, `G3`: các cặp (`G1`,`G2`), (`G2`, `G3`) có sự tương quan cao $\rightarrow$ nếu có học sinh đạt điểm thấp ở đầu kì thì rất có nhiều khả năng giữa kì và cuối kì điểm cũng sẽ thấp theo.

### **Red flags**
**1. Serious data quality concerns:**  

**2. Limitations might affect analysis:**  
